In [ ]:


bigscience/bloom-560m

In [69]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [70]:
bloom = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")
bloom_tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

# bloom_tokenizer.add_bos_token = True  # This is not supported by the tokenizer, so we have to add it manually below

In [64]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import torch
from torch import nn


tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a bos token so we can compute the conditional likelihood of a sentence starting with nothing
tokenizer.add_bos_token = True

model = GPT2LMHeadModel.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)


In [93]:
input_text = 'सोमवार को, स्टैनफ़ोर्ड यूनिवर्सिटी स्कूल ऑफ़ मेडिसिन के वैज्ञानिकों ने एक नए डायग्नोस्टिक उपकरण के आविष्कार की घोषणा की जो कोशिकाओं को उनके प्रकार के आधार पर छाँट सकता है: एक छोटी प्रिंट करने योग्य चिप जिसे स्टैण्डर्ड इंकजेट प्रिंटर का उपयोग करके लगभग एक अमेरिकी सेंट के लिए निर्मित किया जा सकता है.'

bos = bloom_tokenizer(bloom_tokenizer.bos_token, return_tensors='pt')
encoded_input = bloom_tokenizer(input_text, return_tensors='pt')

In [94]:
encoded_input["input_ids"] = torch.concat([bos["input_ids"], encoded_input["input_ids"]], dim=1)
encoded_input["attention_mask"] = torch.concat([bos["attention_mask"], encoded_input["attention_mask"]], dim=1)

encoded_input

{'input_ids': tensor([[     1, 242676,   1452,     15,  12145,   6313,  29190,    398,  15689,
         152936,  36637, 122805,  51943,   1946,   1526,    858, 194876,   1992,
           2122,  40161,  78762,  56124,    398,  10227,    952,  61503,    858,
         216250,   1395,  29124,   1395,   4422,  36557, 121627,   1452,  15399,
          11790,    858,  15338,   1633,   1796, 187418,  14932, 126031,   2122,
          66293, 133986,   1236,   5672,  38806, 136518,  38409,  12145, 224394,
          15689,   1720,  12912,    963,   3861, 133986,  12349,   1944,  22324,
          29731,  34735,   2122,  44709, 109491,    858,   3698,  88813,   4730,
          10356,  14932,    986,     17]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}

In [105]:
log_probs = []

with torch.no_grad():

    for seq_len in range(1, encoded_input["input_ids"].shape[1]):
        _encoded_input = dict(
            input_ids=encoded_input["input_ids"][:, :seq_len],
            attention_mask=encoded_input["attention_mask"][:, :seq_len],
        )

        output = bloom(**_encoded_input)
        log_prob = nn.Softmax(dim=-1)(output.logits)[0][0].log()

        log_probs.append(log_prob[encoded_input["input_ids"][0, seq_len]].item())

In [106]:
sum(log_probs)

-2273.030584335327

In [87]:
nn.Softmax(dim=-1)(output.logits)[0][0].log()

tensor([-23.5910, -16.9164,  -5.5803,  ..., -16.2641, -16.2639, -16.2666],
       grad_fn=<LogBackward0>)